In [1]:
# 데이터 가져오기
import pandas as pd
x_train=pd.read_csv('titanic_x_train.csv', encoding = 'CP949')
x_test=pd.read_csv('titanic_x_test.csv', encoding = 'CP949')
y_train=pd.read_csv('titanic_y_train.csv')

In [2]:
# 전처리하기
x_test_passenger_id =x_test['PassengerId']
x_train = x_train.drop(columns=['PassengerId'])
x_test = x_test.drop(columns=['PassengerId'])
y_train = y_train.drop(columns=['PassengerId'])
x_train = x_train.drop(columns=['티켓번호'])
x_test = x_test.drop(columns=['티켓번호'])
x_train = x_train.drop(columns=['승객이름'])
x_test = x_test.drop(columns=['승객이름'])
x_train = x_train.drop(columns=['나이'])
x_test = x_test.drop(columns=['나이'])
x_train = x_train.drop(columns=['객실번호'])
x_test = x_test.drop(columns=['객실번호'])

In [4]:
#결측치 처리하기
x_train['선착장'] = x_train['선착장'].fillna('S')
x_test['선착장'] = x_test['선착장'].fillna('S')

In [5]:
#인코딩 수행하기
x_train['성별']=x_train['성별'].replace('male',0).replace('female',1)
x_test['성별']=x_test['성별'].replace('male',0).replace('female',1)
선착장_dummy = pd.get_dummies(x_train['선착장'],drop_first = True).rename(columns= {'Q':'선착장Q', 'S':'선착장S'})
x_train = pd.concat([x_train,선착장_dummy],axis=1)
x_train=x_train.drop(columns=['선착장'])
선착장_dummy = pd.get_dummies(x_test['선착장'],drop_first = True).rename(columns= {'Q':'선착장Q', 'S':'선착장S'})
x_test = pd.concat([x_test,선착장_dummy],axis=1)
x_test=x_test.drop(columns=['선착장'])

In [6]:
# 파생변수 만들기
x_train['가족수'] =x_train['형제자매배우자수']+x_train['부모자식수']
x_train=x_train.drop(columns=['형제자매배우자수','부모자식수'])
x_test['가족수'] =x_test['형제자매배우자수']+x_test['부모자식수']
x_test=x_test.drop(columns=['형제자매배우자수','부모자식수'])

In [7]:
#데이터 분리하기
from sklearn.model_selection import train_test_split
X_TRAIN,X_TEST,Y_TRAIN,Y_TEST = \
train_test_split(x_train,y_train, test_size=0.2 ,random_state=10)

In [9]:
# 모델 학습하기
from xgboost import XGBClassifier
model = XGBClassifier(n_estimators=100, max_depth = 5 , eval_metric = 'error',\
                     random_state=10)
model.fit(X_TRAIN,Y_TRAIN)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              eval_metric='error', gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=5, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=4,
              num_parallel_tree=1, predictor='auto', random_state=10,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [11]:
# 최종 결과를 파일로 저장하기
y_test_predicted = pd.DataFrame(model.predict(x_test)).rename(columns={0:'Survived'})
final=pd.concat([x_test_passenger_id,y_test_predicted],axis=1)
final.to_csv('123456.csv',index=False)